# Feature extractor



Рассмотрим карты признаков предпоследнего слоя нейронной сети. В архитектуре **AlexNet** предпоследний слой является полносвязным, поэтому его активации представляют собой вектор.

Когда изображение подается на вход сети, оно кодируется в виде массива данных размером $150528$ чисел ($224\times224\times3 = 150528$), представляющих пиксели изображения. После прохождения через сеть на выходе мы получаем вектор, состоящий из $4096$ чисел.

Этот процесс можно рассматривать как понижение размерности данных примерно в 36 раз. Несмотря на уменьшение размерности, полученный вектор содержит достаточную информацию для решения задачи классификации на 1000 классов.

Подобные векторы признаков называются **embedding** и широко применяются в различных задачах машинного обучения и компьютерного зрения.

**Последний слой**

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/feature_extractor.png" width="1000">

Чтобы убедиться в полезности полученных представлений, кластеризуем их при помощи [k-nearest neighbors 📚[wiki]](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) алгоритма.




**Последний слой: ближайшие соседи**

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L06/last_layer_knn.jpg" width="900"></center>

<center><em>Source: <a href="https://cs231n.stanford.edu/slides/2023/lecture_13.pdf">CS231: Self-Supervised Learning</a></em></center>

In [ ]:
from torchvision import models

alexnet = models.alexnet(weights="AlexNet_Weights.DEFAULT")
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Чтобы получить embedding изображения, отключим последний слой. Выведем структуру модели, чтобы найти его:

In [ ]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

И заменим его пустышкой `nn.Identity()` — класс, который возвращает вход без изменений:

In [ ]:
alexnet.classifier[6] = nn.Identity()

Загрузим датасет:

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor, Resize, Normalize, Compose
from torch.utils.data import DataLoader, random_split

torch.manual_seed(42)

transform = Compose(
    [Resize(224), ToTensor(), Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)

testset = CIFAR10(root="./CIFAR10", train=False, download=True, transform=transform)
train, test, _ = random_split(testset, [512, 128, 10000 - 512 - 128])
train_loader = DataLoader(train, batch_size=128, shuffle=False, drop_last=True)

Files already downloaded and verified


Добавим функцию, которая сохраняет выходы измененной модели в массиве, а также возвращает метки классов:

In [ ]:
from tqdm import tqdm


def get_embeddings(loader):
    embeddings = []
    labels = []
    for img, label in tqdm(loader):
        emb = alexnet(img)
        embeddings.append(emb.detach())
        labels.append(label)
    embeddings = torch.stack(embeddings).reshape(-1, 4096).numpy()
    labels = torch.stack(labels).flatten().numpy()
    return embeddings, labels

Превратим картинки в векторы признаков:

In [ ]:
%%time
x, y = get_embeddings(train_loader)

100%|██████████| 4/4 [00:18<00:00,  4.59s/it]

CPU times: user 16.4 s, sys: 1.38 s, total: 17.8 s
Wall time: 18.4 s


Теперь у нас есть $512$ векторов по $4096$ значения в каждом и $512$ меток классов:

In [ ]:
print(x.shape, y.shape)

(512, 4096) (512,)


"Обучим" на них k-NN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x, y)

KNeighborsClassifier()

Получим векторы признаков (embeddings) для тестовых картинок:

In [ ]:
%%time

test_loader = DataLoader(test, batch_size=32, shuffle=False, drop_last=True)
test_emb, gt_labels = get_embeddings(test_loader)

100%|██████████| 4/4 [00:04<00:00,  1.02s/it]

CPU times: user 4.02 s, sys: 117 ms, total: 4.14 s
Wall time: 4.12 s


Получаем предсказания и считаем accuracy:

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = neigh.predict(test_emb)

accuracy = accuracy_score(gt_labels, y_pred)
print("k-NN accuracy", accuracy)

k-NN accuracy 0.4609375


Как видим, активации на последних слоях сети достаточно информативны.

# Transfer learning

Как обучить нейросеть  на своих данных, когда их мало?

Можно взять обученную модель, заменить у нее несколько последних слоев и обучить только их.

Ранее обученные слои будут извлекать признаки (feature extractor), и полученные таким образом представления (embedding) будут классифицироваться вновь добавленными слоями.

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/transfer_learning_change_classes_scheme.png" width="700">

Возможно, не все фильтры модели будут использованы эффективно на новой задаче. К примеру, если мы работаем с изображениями, связанными только с едой, не все фильтры на скрытых слоях предобученной на ImageNet модели окажутся полезны для нашей задачи.

Поэтому **после того, как был обучен новый классификатор**, можно дообучить все или некоторые промежуточные слои. При этом используют меньший learning rate, чем при обучении нейросети с нуля: мы знаем, что по крайней мере часть весов нейросети выполняет свою задачу хорошо, и не хотим испортить это быстрыми изменениями.

Поэтому такой подход называется  **fine-tuning** — мы дополнительно "настраиваем" и промежуточные слои под нашу задачу.

Можно делать настройку постепенно: сначала учить только последние добавленные нами слои сети, затем самые близкие к ним, и после этого учить уже все веса нейросети вместе.

Иногда fine-tuning считается синонимом Transfer learning, в этом случае часть от предтренированной сети называют **backbone** ("позвоночник"), а добавленную часть — **head** ("голова").



## Шаг 1. Получение предварительно обученной модели

Последовательно рассмотрим шаги, необходимые для реализации подхода transfer learning.

Первым шагом является выбор предварительно обученной модели, которую мы хотели бы использовать в качестве основы для обучения. Основным предположением является то, что признаки, которые умеет выделять из данных предобученная модель, хорошо подойдут для решения нашей частной задачи. Поэтому эффект от Transfer learning будет тем лучше, чем более схожими будут **домены** в нашей задаче и в задаче, на которой предварительно обучалась модель.

Для задач обработки изображений очень часто используются модели, предобученные на ImageNet. Такой подход распространен, однако если ваша задача связана, например, с обработкой снимков клеток под микроскопом, то модель, предобученная на более близком домене (тоже на снимках клеток, пусть и совсем других), может быть лучшим начальным решением.

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/transfer_learning_step_1.png" width="600">

In [ ]:
from torchvision import models

model = models.alexnet(weights="AlexNet_Weights.DEFAULT")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:05<00:00, 41.2MB/s]


## Шаг 2. Заморозка предобученных слоев

Мы предполагаем, что первые слои модели уже хорошо натренированы выделять важные признаки из данных, и мы не хотим их "сломать".

Если начнем дообучать их вместе с новыми слоями, которые инициализируются случайно, то на первых шагах обучения ошибка будет большой и мы можем сильно изменить "хорошие" предобученные веса.

Поэтому требуется "заморозить" предобученные веса. На практике заморозка означает **отключение подсчета градиентов**. Таким образом, при последующем обучении параметры с отключенным подсчетом градиентов не будут обновляться.

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/transfer_learning_step_2.png" width="600">

In [ ]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

## Шаг 3. Добавление новых обучаемых слоев

В отличие от начальных слоев, которые выделяют достаточно общие признаки из данных, более близкие к выходу слои предобученной модели сильно специфичны конкретно под ту задачу, на которую она обучалась. Для моделей, предобученных на ImageNet, последний слой заточен конкретно под предсказание 1000 классов из этого набора данных. Кроме этого, последние слои могут не подходить под новую задачу архитектурно: в новой задаче может быть меньше классов, 10 вместо 1000. Поэтому требуется **заменить последние один или несколько слоев** предобученной модели на новые, подходящие под нашу задачу. При этом, естественно, веса в этих слоях будут инициализированы случайно. Именно эти слои мы и будем обучать на следующем шаге.




<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/transfer_learning_step_3.png" width="600">

In [ ]:
print(model.classifier)

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [ ]:
from torch import nn

model.classifier[6] = nn.Linear(4096, 10, bias=True)  # For CIFAR

Убедимся, что обучаться будет только вновь добавленный слой:

In [ ]:
for name, param in model.named_parameters():
    print(name, "\t", param.requires_grad)

features.0.weight 	 False
features.0.bias 	 False
features.3.weight 	 False
features.3.bias 	 False
features.6.weight 	 False
features.6.bias 	 False
features.8.weight 	 False
features.8.bias 	 False
features.10.weight 	 False
features.10.bias 	 False
classifier.1.weight 	 False
classifier.1.bias 	 False
classifier.4.weight 	 False
classifier.4.bias 	 False
classifier.6.weight 	 True
classifier.6.bias 	 True


## Шаг 4. Обучение новых слоев

Все, что нам теперь нужно, — обучить новые слои на наших данных. При этом замороженные слои используются лишь как **экстрактор высокоуровневых признаков**.
Обучение такой модели существенно ничем не отличается от обучения любой другой модели: используются обучающая и валидационная выборки, контролируется изменение функции потерь и функционала качества.

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/transfer_learning_step_4.png" width="600">

Загрузим датасет:

In [ ]:
import torch
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor, Resize, Normalize, Compose
from torch.utils.data import DataLoader, random_split

torch.manual_seed(42)

transform = Compose(
    [Resize(224), ToTensor(), Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)

testset = CIFAR10(root="./CIFAR10", train=False, download=True, transform=transform)
train, test, _ = random_split(testset, [512, 128, 10000 - 512 - 128])
train_loader = DataLoader(train, batch_size=128, shuffle=False, drop_last=True)

100%|██████████| 170498071/170498071 [00:05<00:00, 29927942.80it/s]


Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10


И напишем функцию для обучения:

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def train_model(model, num_epochs=1, lr=1e-3):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=lr)
    for epoch in range(num_epochs):
        for imgs, labels in tqdm(train_loader):
            optimizer.zero_grad()  # Clean existing gradients
            outputs = model(imgs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()  # Backpropagate the gradients
            optimizer.step()
        print(f"\nEpoch {epoch} Loss {loss.item()}")

In [ ]:
train_model(model, 5)  # train only last layer

100%|██████████| 4/4 [00:19<00:00,  4.88s/it]



Epoch 0 Loss 2.1474616527557373


100%|██████████| 4/4 [00:17<00:00,  4.41s/it]



Epoch 1 Loss 1.8127602338790894


100%|██████████| 4/4 [00:18<00:00,  4.51s/it]



Epoch 2 Loss 1.3982957601547241


100%|██████████| 4/4 [00:18<00:00,  4.59s/it]



Epoch 3 Loss 1.1296778917312622


100%|██████████| 4/4 [00:18<00:00,  4.71s/it]


Epoch 4 Loss 0.9322646856307983


В модели могут быть слои, поведение которых различно при выводе (inference) и обучении. Например, слои Dropout и Batchnorm.

Если такие слои в модели есть, то их следует перевести в inference режим, используя метод `nn.Module.eval()`

## Шаг 5. Тонкая настройка модели (fine-tuning)

После того, как мы обучили новые слои модели, и они уже как-то решают задачу, мы можем разморозить ранее замороженные веса, чтобы **тонко настроить** их под нашу задачу в надежде, что это позволит еще немного повысить качество.

Нужно быть осторожным на этом этапе: использовать learning rate на порядок или два меньший, чем при основном обучении, и одновременно с этим следить за возникновением переобучения. Переобучение при fine-tuning может возникать из-за того, что мы резко увеличиваем количество настраиваемых параметров модели, но при этом наш датасет остается небольшим, и мощная модель может начать заучивать обучающие данные.

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/transfer_learning_step_5.png" width="600">

In [ ]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = True

In [ ]:
%%time
train_model(model, num_epochs=3, lr=1e-5)  # fine tune all layers

100%|██████████| 4/4 [00:52<00:00, 13.01s/it]



Epoch 0 Loss 0.8834771513938904


100%|██████████| 4/4 [00:47<00:00, 11.80s/it]



Epoch 1 Loss 0.9074606895446777


100%|██████████| 4/4 [00:53<00:00, 13.40s/it]


Epoch 2 Loss 0.9113494753837585
CPU times: user 2min 14s, sys: 9.65 s, total: 2min 24s
Wall time: 2min 32s


Проверим accuracy:

In [ ]:
test_loader = DataLoader(test, batch_size=32, shuffle=False, drop_last=True)
y_true = []
y_pred = []
for imgs, labels in test_loader:
    outputs = model(imgs.to(device))
    y_true.append(labels.numpy())
    preds = outputs.argmax(dim=1)
    y_pred.append(preds.detach().cpu().numpy())

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score


y_true = np.stack(y_true).flatten()
y_pred = np.stack(y_pred).flatten()

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy : {accuracy:.2f}")

Accuracy : 0.52
